## Exercise 2
**3.** Implement a class MLEClassifier that works on categorical problems (X and Y discrete)
and predicts classes based on the estimated posterior probabilities.

In [1]:
# imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import random
from tqdm import tqdm

In [4]:
class MLEClassifier:
    def train(self, X, Y):
        self.X = X
        self.Y = Y
        self.dict = {}
        for x, y in zip(self.X, self.Y):
            temp = ",".join(x)
            if temp not in self.dict.keys():
                self.dict[temp] = {}
            newDict = self.dict[temp]
            if y not in newDict.keys():
                newDict[y] = 0
            newDict[y] = newDict[y]+1
        for key in self.dict.keys():
            sum = 0
            for newKey in self.dict[key]:
                sum = sum+self.dict[key][newKey]
            for newKey in self.dict[key]:
                self.dict[key][newKey] /= sum

    def predict(self, X):
        predict = ",".join(X)
        print(max(self.dict[predict], key=self.dict[predict].get))


# Read dataset
df = pd.read_csv('breast-cancer.csv').to_numpy()
model = MLEClassifier()
X = df[:, :2]
Y = df[:, -1]
model.train(X, Y)
model.predict(["'40-49'", "'premeno'"])


'no-recurrence-events'


**4.** Implement a class BayesianCategoricalClassifier. It should receive a parameter
is_naive at initialization time to specify whether it should behave in a naive way.
Models may be learned without the adjusted frequencies.

In [5]:
class BayesianCategoricalClassifier:
    def __init__(self,is_naive):
        self.is_naive=is_naive
    def train(self,X,Y):
        classes = np.unique(Y)
        self.m=set()
        self.n=set()
        self.Di={}
        self.table = {}
        self.pi={}
        self.ni={}
        if not (self.is_naive):
            for k in classes:
                self.Di[k]=[]
                for x,y in zip(X,Y):
                    if(y==k):
                        self.Di[k].append(x)
                for di in self.Di:
                    self.ni[di]=len(self.Di[di])
                    self.pi[di]=len(self.Di[di])/len(X)
                    self.table[k] ={}
                    for x in self.Di[di]:
                        self.m.add(x[0])
                        self.n.add(x[1])
                        if not x[0] in self.table[k]:
                            self.table[k][x[0]]={}
                        if not x[1] in self.table[k][x[0]]:
                            self.table[k][x[0]][x[1]]=0
                        self.table[k][x[0]][x[1]]+=1/self.ni[di]
        else:
            for k in classes:
                self.Di[k]=[]
                for x,y in zip(X,Y):
                    if(y==k):
                        self.Di[k].append(x)
                for di in self.Di:
                    self.ni[di]=len(self.Di[di])
                    self.pi[di]=len(self.Di[di])/len(df)
                    self.table[k] ={}
                    for x in self.Di[di]:
                        self.m.add(x[0])
                        if not "x1"+x[0] in self.table[k]:
                            self.table[k]["x1"+x[0]]=0
                        if not "x2"+x[1] in self.table[k]:
                            self.table[k]["x2"+x[1]]=0
                        self.table[k]["x1"+x[0]]+=1/self.ni[di]
                        self.table[k]["x2"+x[1]]+=1/self.ni[di]

    def predict(self,X):
        res = {}
        if not self.is_naive:
            for k in self.Di.keys():
                try:
                    res[k]=self.pi[k]*self.table[k][X[0]][X[1]]
                except KeyError:
                    res[k]=self.pi[k]*(1/(self.ni[k]+len(self.n)*len(self.m)))
        else:
            for k in self.Di.keys():
                try:
                    res[k]=self.pi[k]*self.table[k]["x1"+X[0]]*self.table[k]["x2"+X[1]]
                except KeyError:
                    res[k]=self.pi[k]*1/(self.ni[k]+len(self.m)) * self.table[k]["x2"+X[1]]
        return max(res, key=res.get)


df = pd.read_csv('iris.csv')
df = df[["sepal_length", 'sepal_width', 'species']]
conditions = [
    (df['sepal_length'] >= 4.3 )&( df['sepal_length'] <= 5.2),
    (df['sepal_length'] > 5.2 )&( df['sepal_length'] <= 6.1),
    (df['sepal_length'] > 6.1 )&( df['sepal_length'] <= 7.0),
    (df['sepal_length'] > 7.0 )&( df['sepal_length'] <= 7.9),
    ]
values = ['Very Short', 'Short', 'Long', 'Very Long']
df['sepal_length_dis'] = np.select(conditions, values)
conditions = [
    (df['sepal_width'] >= 2.0 )&( df['sepal_width'] <= 2.8),
    (df['sepal_width'] > 2.8 )&( df['sepal_width'] <= 3.6),
    (df['sepal_width'] > 3.6 )&( df['sepal_width'] <= 4.4),
    ]
values = ['Short', 'Medium', 'Long',]
df['sepal_width_dis'] = np.select(conditions, values)
conditions = [
    (df['species'] == 'setosa' ),
    (df['species'] != 'setosa' ),
]
values = ['setosa', 'others',]
df['species_dis'] = np.select(conditions, values)

df = df[["sepal_length_dis", 'sepal_width_dis', 'species_dis']].to_numpy()
X=df[:,:2]
Y=df[:,2]

model = BayesianCategoricalClassifier(False)
model.train(X,Y)

test = ["Long","Long"]

model.predict(test)

'others'